In [ ]:
pygui(false)
PyPlot.svg(true)

import pbc
import lj
import ns

NS Simulation
===========

In [ ]:
dim = 2;
Nat = 7;
Nw = 10;
Nsteps = 8;
Niter = Nw*32;

#lj.set_shiftedLJ();
pbc.set_density(Nat, 0.00925925);

In [ ]:
#Profile.clear()
#@profile

xs = zeros(dim, Nat, Nw)
xhist = zeros(Niter, dim*Nat)

E = zeros(Niter, 1)
for Nat=7:7
    pbc.set_density(Nat, 0.00925925);
    (xhist, E[:,Nat-6], xs, accept_ratio_history, stepsize_history) = ns.nestedsampling(lj, dim, Nat, Nw, Niter, Nsteps; seed=1, extrawalks=false, walksource=false)
end
#using ProfileView
#ProfileView.view()

configuration plots
====================

In [ ]:
import pbc.L

function plot_configs(configs)
    
    b = 5;
    if L != 0.0
        bshift = L*1.1;
    else
        bshift = b;
    end

    figure()
    i=0
    for c in configs
        i=i+1
        y0 = -div(i-1,b)*bshift; 
        x0 = mod(i-1,b)*bshift;
    
        if L != 0.0
            x = c; 
            x[1,:] -= x[1,1];
            x[2,:] -= x[2,1];
            pbc.map_into_cell!(x);
            plot(x[1,:]+x0, x[2,:]+y0, ".")
            plot([x0-0.5*L, x0-0.5*L, x0+0.5*L, x0+0.5*L, x0-0.5*L], [y0-0.5*L, y0+0.5*L, y0+0.5*L, y0-0.5*L, y0-0.5*L], "k-");
        else
            plot(c[1,:]+x0, c[2,:]+y0, ".")
            plot(x0, y0, "ko", markerfacecolor="none");
        end
    end
    axis("scaled"); axis("off");
    tight_layout()
    close(gcf())
end

In [ ]:
configs = Any[]
for i=1:20:Niter
    push!(configs, reshape(xhist[i,:], (dim,Nat)))
end
plot_configs(configs)

In [ ]:
configs = Any[]
for i = 1:size(xs,3)
    push!(configs, xs[:,:,i])
end
plot_configs(configs)

Thermodynamics plots
==============

In [ ]:

figure();
T=[0.005:0.001:0.2];


(Z,U,Cv) = ns.thermo(T, E, Nat, Nw);
plot(4*T, Cv/Nat, "b", label="LJ $Nat");

plot([minimum(4*T), maximum(4*T)], [1.5, 1.501], "k:")

xlabel("T"); ylabel(L"C_v");
legend();
close(gcf())